# Gold Customer Analysis

Exploratory notebook analyzing the gold-layer customer models (`gold.gld_customer_analytics`, `gold.gld_revenue_summary`).

In [1]:
import pandas as pd
import plotly.express as px
from helpers import connect, query_df

con = connect(read_only=True)
con

## Portfolio Snapshot

In [2]:
customer_summary = query_df(
    con,
    '''
    SELECT
        COUNT(*) AS total_customers,
        SUM(CASE WHEN customer_segment = 'VIP' THEN 1 ELSE 0 END) AS vip_customers,
        SUM(CASE WHEN total_orders > 1 THEN 1 ELSE 0 END) AS repeat_buyers,
        ROUND(SUM(lifetime_value), 2) AS lifetime_revenue,
        ROUND(AVG(avg_order_value), 2) AS avg_order_value
    FROM gold.gld_customer_analytics
    '''
)
customer_summary

,total_customers,vip_customers,repeat_buyers,lifetime_revenue,avg_order_value
0,28,10.0,25.0,20444.35,243.09


## Segment Mix

In [3]:
segment_mix = query_df(
    con,
    '''
    SELECT
        customer_segment,
        COUNT(*) AS customers,
        ROUND(SUM(lifetime_value), 2) AS revenue,
        ROUND(AVG(avg_order_value), 2) AS avg_order_value
    FROM gold.gld_customer_analytics
    GROUP BY 1
    ORDER BY revenue DESC
    '''
)
segment_mix

,customer_segment,customers,revenue,avg_order_value
0,VIP,10,10718.66,334.71
1,Loyal,11,8420.38,255.16
2,Active,4,1305.31,163.16
3,Prospect,3,0.00,0.00


In [4]:
fig_segment = px.bar(
    segment_mix,
    x='customer_segment',
    y='revenue',
    text='customers',
    title='Revenue + Customer Count by Segment',
    labels={'customer_segment': 'Segment', 'revenue': 'Lifetime Revenue'}
)
fig_segment.update_traces(textposition='outside')
fig_segment

## Recency Health

In [5]:
recency_mix = query_df(
    con,
    '''
    SELECT
        recency_segment,
        COUNT(*) AS customers,
        ROUND(SUM(lifetime_value), 2) AS revenue
    FROM gold.gld_customer_analytics
    GROUP BY 1
    ORDER BY customers DESC
    '''
)
recency_mix

,recency_segment,customers,revenue
0,Dormant,25,20444.35
1,Never Ordered,3,0.00


In [6]:
fig_recency_customers = px.bar(
    recency_mix,
    x='recency_segment',
    y='customers',
    title='Customers by Recency Segment',
    labels={'recency_segment': 'Recency Segment', 'customers': 'Customers'}
)
fig_recency_revenue = px.bar(
    recency_mix,
    x='recency_segment',
    y='revenue',
    title='Revenue by Recency Segment',
    labels={'recency_segment': 'Recency Segment', 'revenue': 'Lifetime Revenue'},
    color_discrete_sequence=['#ff7f0e']
)
fig_recency_customers, fig_recency_revenue

(Figure({
     'data': [{'hovertemplate': 'Recency Segment=%{x}<br>Customers=%{y}<extra></extra>',
               'legendgroup': '',
               'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
               'name': '',
               'orientation': 'v',
               'showlegend': False,
               'textposition': 'auto',
               'type': 'bar',
               'x': array(['Dormant', 'Never Ordered'], dtype=object),
               'xaxis': 'x',
               'y': {'bdata': 'GQM=', 'dtype': 'i1'},
               'yaxis': 'y'}],
     'layout': {'barmode': 'relative',
                'legend': {'tracegroupgap': 0},
                'template': '...',
                'title': {'text': 'Customers by Recency Segment'},
                'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Recency Segment'}},
                'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Customers'}}}
 }),
 Figure({
     'data': [{'hovertemplate': 'Recency

## Acquisition Sources

In [7]:
acquisition_mix = query_df(
    con,
    '''
    SELECT
        acquisition_source,
        COUNT(*) AS customers,
        ROUND(SUM(lifetime_value), 2) AS revenue,
        ROUND(AVG(avg_order_value), 2) AS avg_order_value
    FROM gold.gld_customer_analytics
    GROUP BY 1
    ORDER BY revenue DESC
    '''
)
acquisition_mix

,acquisition_source,customers,revenue,avg_order_value
0,organic,11,8199.56,245.97
1,referral,7,5962.63,271.63
2,paid_search,5,3394.16,239.51
3,social_media,5,2888.00,200.38


In [8]:
fig_acquisition = px.bar(
    acquisition_mix,
    x='acquisition_source',
    y='revenue',
    text='customers',
    title='Revenue by Acquisition Source',
    labels={'acquisition_source': 'Source', 'revenue': 'Lifetime Revenue'}
)
fig_acquisition.update_traces(textposition='outside')
fig_acquisition

## Geographic Concentration

In [9]:
geo_mix = query_df(
    con,
    '''
    SELECT
        country_code,
        COUNT(*) AS customers,
        ROUND(SUM(lifetime_value), 2) AS revenue
    FROM gold.gld_customer_analytics
    GROUP BY 1
    ORDER BY revenue DESC
    LIMIT 15
    '''
)
geo_mix

,country_code,customers,revenue
0,US,7,5085.04
1,IN,2,1977.94
2,DE,2,1361.29
3,SPAIN,1,1178.94
4,MEXICO,1,1102.73
5,FR,1,1088.93
6,SOUTH KOREA,1,1075.97
7,IRELAND,1,1063.14
8,AU,1,1031.95
9,SINGAPORE,1,973.66


In [10]:
fig_geo = px.bar(
    geo_mix,
    x='country_code',
    y='revenue',
    text='customers',
    title='Top Markets by Lifetime Revenue',
    labels={'country_code': 'Country', 'revenue': 'Lifetime Revenue'}
)
fig_geo.update_traces(textposition='outside')
fig_geo

## Cohort & Activity Trends

In [11]:
cohort_growth = query_df(
    con,
    '''
    SELECT
        DATE_TRUNC('month', first_order_date) AS cohort_month,
        COUNT(*) AS new_customers,
        ROUND(SUM(lifetime_value), 2) AS lifetime_revenue
    FROM gold.gld_customer_analytics
    GROUP BY 1
    ORDER BY 1
    '''
)
cohort_growth

,cohort_month,new_customers,lifetime_revenue
0,2024-02-01,9,8002.29
1,2024-03-01,10,7249.67
2,2024-04-01,4,3809.19
3,2024-05-01,2,1383.20
4,NaT,3,0.00


In [12]:
fig_cohort = px.line(
    cohort_growth,
    x='cohort_month',
    y='new_customers',
    markers=True,
    title='New Customers by Cohort Month',
    labels={'cohort_month': 'Cohort Month', 'new_customers': 'Customers'}
)
fig_cohort

In [13]:
fig_cohort_rev = px.bar(
    cohort_growth,
    x='cohort_month',
    y='lifetime_revenue',
    title='Lifetime Revenue by Cohort Month',
    labels={'cohort_month': 'Cohort Month', 'lifetime_revenue': 'Lifetime Revenue'}
)
fig_cohort_rev

## Top Customers

In [14]:
top_customers = query_df(
    con,
    '''
    SELECT
        full_name,
        country_code,
        customer_segment,
        total_orders,
        ROUND(lifetime_value, 2) AS lifetime_value,
        ROUND(avg_order_value, 2) AS avg_order_value,
        last_order_date
    FROM gold.gld_customer_analytics
    ORDER BY lifetime_value DESC
    LIMIT 25
    '''
)
top_customers

,full_name,country_code,customer_segment,total_orders,lifetime_value,avg_order_value,last_order_date
0,Noah Garcia,SPAIN,VIP,3,1178.94,392.98,2024-10-20
1,Ivy Martinez,MEXICO,VIP,3,1102.73,367.58,2024-10-10
2,Xavier Dupont,FR,VIP,3,1088.93,362.98,2024-12-20
3,Wendy Kim,SOUTH KOREA,VIP,3,1075.97,358.66,2024-11-15
4,Zane O'brien,IRELAND,VIP,3,1063.14,354.38,2024-10-05
5,Riley Kumar,IN,VIP,3,1059.77,353.26,2024-11-10
6,Alice Johnson,US,VIP,5,1047.78,209.56,2024-12-10
7,Hannah Berg,DE,VIP,3,1035.76,345.25,2024-11-05
8,Frank Miller,US,VIP,4,1033.69,258.42,2024-11-01
9,Emma Davis,AU,VIP,3,1031.95,343.98,2024-09-01
